In [ ]:
import yaml
import time
import random
import requests
def get_headers(config):
    headers = {
        "Cookie": config['cookie'],
        "User-Agent": config['user-agent']
    }
    return headers
def get_params(config):
    url = "https://mp.weixin.qq.com/cgi-bin/appmsgpublish"
    begin = "0"
    params = {
        "sub": "list",
        "search_field":"null",
        "sub_action": "list_ex", 
        "begin": begin,
        "free_publish_type": "1",
        "count": "5",
        "fakeid": config['fakeid'],
        "type": "101_1",
        "token": config['token'],
        "lang": "zh_CN",
        "f": "json",
        "ajax": "1"
    }
    return params
def get_article_list(headers, params):
    i = 0
    url = "https://mp.weixin.qq.com/cgi-bin/appmsgpublish"
    column_name = "aid,appmsgid,author_name,title,cover_img,digest,link,create_time"
    article_list_path = "article_list2.csv"
    with open(article_list_path, "a") as f:
        f.write(column_name + '\n')
    while True:
        begin = i * 5
        params["begin"] = str(begin)
        # 随机暂停几秒，避免过快的请求导致过快的被查到
        time.sleep(random.randint(1,10))
        resp = requests.get(url, headers=headers, params = params, verify=False)
        # 微信流量控制, 退出
        if resp.json()['base_resp']['ret'] == 200013:
            print("frequencey control, stop at {}".format(str(begin)))
            time.sleep(3600)
            continue
        
        if i == "0":
            total_count = eval(resp.json()['publish_page'])['total_count']
            print("We have "+str(tatal_count) + " articles.")
        publish_list = eval(resp.json()['publish_page'])['publish_list']
        # 如果返回的内容中为空则结束
        if len(publish_list) == 0:
            print("all ariticle parsed")
            break
    
        for publish in publish_list:
            publish = eval(publish['publish_info'].replace("true","True").replace("false","False"))['appmsgex'][0]
            info = '"{}","{}","{}","{}","{}","{}","{}","{}"'.format(str(publish["aid"]), \
                str(publish['appmsgid']), str(publish['author_name']), \
                str(publish['title'].replace("\n","").replace(",",";")), \
                str(publish['cover']), str(publish['digest'].replace("\n","").replace(",",";")), \
                str(publish['link']), str(publish['create_time']))
            with open(article_list_path, "a") as f:
                f.write(info+'\n')
            print("\n".join(info.split(",")))
            print("\n\n---------------------------------------------------------------------------------\n")

        # 翻页
        i += 1
def main():
    with open("chaos-gravity.yaml", "r") as file:
        file_data = file.read()
    config = yaml.safe_load(file_data) 
    headers = get_headers(config)
    params = get_params(config)
    get_article_list(headers, params)
if __name__ == '__main__':
    main()

In [ ]:
import pandas as pd
from newspaper import Article
import html2text as ht
import yaml
import time
import requests
from bs4 import BeautifulSoup
import os
import tomd
import re

with open("chaos-gravity.yaml", "r") as file:
    file_data = file.read()
config = yaml.safe_load(file_data)
def get_headers(config):
    headers = {
        "Cookie": config['cookie'],
        "User-Agent": config['user-agent']
    }
    return headers

pd.set_option('max_colwidth', 1000)
article_list = pd.read_csv('article_list2.csv', encoding='GB18030')
headers = get_headers(config)

    
def download_images(soup, content):
    dir_name = "assets/images/" + str(filename[:-3]) + "/"
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    cnt = 1
    images = content.find_all("img")
    for image in images:
        img_src = image.get('data-src')
        img_type = image.get('data-type')
        img_name = "{0:d}.{1:s}".format(cnt, img_type if img_type else 'png')
        cnt += 1
        file_path = "assets/images/{0:s}/{1:s}".format(filename[:-3], img_name)
        if not os.path.exists(file_path):
            with open (file_path, 'wb') as file:
                response = requests.get(url = img_src)
                for block in response.iter_content(1024):
                    if block:
                        file.write(block)
                    else:
                        break
        tag = soup.new_tag('span')
        tag.string = "![](assets/images/{0:s}/{1:s})".format(filename[:-3], img_name)
        image.replace_with(tag)

        
def download_videos(soup, content):
    dir_name = "assets/videos/" + str(title) + "/"
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    if videos:
        for video in videos:
            video_src = video.get('data-src')

            tag = soup.new_tag('span')
            tag.string = "[此处是视频]({0:s})".format(video_src)
            video.replace_with(tag)

def for_list(content):
    for a in content.find_all("ol"):
        i = 1
        for b in a.find_all("li"):
            tag = soup.new_tag('span')
            tag.string = "{0:d}. {1:s}\n".format(i, b.text)
            b.replace_with(tag)
            i = i+1
def for_code2(content):
    for a in content.find_all(class_="code-snippet__fix code-snippet__js"):
        for b in a.find_all("code"):
            tag = soup.new_tag('code')
            tag.string = "\n" + b.text
            b.replace_with(tag)            
def for_code(content):
    for a in content.find_all(class_="code-snippet__fix code-snippet__js"):
        tag = soup.new_tag('code')
        string = ""
        length = len(a.find_all("code"))
        i = 1
        for b in a.find_all("code"):
            string = string + "\n" + b.text
            if i<length:
                b.extract()
            i = i+1
        tag.string = string
        b.replace_with(tag)
for index, row in article_list.iterrows():
    print(index)
    print(row)
    create_time = time.strftime("%Y-%m-%d",time.localtime(row["create_time"]))
    url = row['link'].replace("\/","/")
    article = requests.get(url,headers=headers)
    if article.status_code == 200:
        html = article.text
    # for_code 
    html = html.replace("<br  />","\n")
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find(id='img-content')
    #有些文章已经被删除了，就提取不到content了
    if content:
        title = content.find(class_='rich_media_title').text.strip()
        print(title)
        # \/:*?"<>|这些字符不能作为文件名或者文件夹名
        filename = title.replace('\/','-').replace('\\','-').replace('\"','-').replace('|','-').replace(":","_").replace("?","_").replace("*","_").replace("<","_").replace(">","_") + '.md'
        try:
            copyright = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta icon_appmsg_tag appmsg_title_tag weui-wa-hotarea').text
        except:
            copyright = None
        try:
            author = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta_link weui-wa-hotarea').text
        except:
            author = content.find(class_ = "rich_media_meta rich_media_meta_text").text.split("\n")[-2].split(" ")[-1]
        blog_name = content.find(class_='profile_nickname').text
        blog_sign = content.div.div.div.find_all('p')[1].span.text
        for i in soup.html.body.find_all('script'):
            if 'publish_time' in str(i):
                publish_time = int(i.text.split("document.getElementById")[0].split("{e(")[-1].split(",\"")[-1].split("\",")[0])
        content.find(id='meta_content').decompose()
        download_images(soup, content)
        brs = content.find_all('br')
        if brs:
            for br in brs:
                br.decompose()
        for_list(content)
        for_code2(content)
        mdText = str(content).replace('<br/>','').replace('</code><code>', "").replace('\n</code>','</code>').replace('<code>\n','<code>')
        # 代码部分自动换行
        mdText = tomd.Tomd(mdText).markdown
        # 改格式转错的地方
        mdText = mdText.replace('# \n            \n'+title, '## '+title).replace("# \n", "\n")
        mdText = re.sub(r'\n[\- ]+[ ]+\n```', "\n \n```", mdText)
        with open(filename, 'w', encoding='utf8') as file:
            file.write(mdText)

In [ ]:
# 去掉<code> </code>  <em> </em> 将'-&gt;'  替换成‘->’ 
import os 
for item in os.listdir():
    if item.split(".")[-1] == "md":
        with open(item, 'r', encoding='utf8') as file:
            print("-------------------------------------------------")
            str = ''
            a = str.join(file.readlines())
            if ('</em>' in a):
                print(item)

In [ ]:
#查看是否有丢失图片，或者排序错误
import os 
for item in os.listdir():
    if item.split(".")[-1] == "md":
        with open(item, 'r', encoding='utf8') as file:
            print("-------------------------------------------------")
            print(item)
            str = ''
            a = str.join(file.readlines())
            a = a.split("assets/images/")
            if len(a)>1:
                a = a[1:]
                i = 1
                for b in a:
                    b = b.split("\n")[0]
                    b = b.split("/")[1].split(".")[0]
            
                    if i !=int(b):
                        print("No")
                    i = i+1

In [ ]:
#改成Jekyll格式
import pandas as pd
from newspaper import Article
import html2text as ht
import yaml
import time
import requests
from bs4 import BeautifulSoup
import os
import tomd
import re

with open("../wechat/chaos-gravity.yaml", "r") as file:
    file_data = file.read()
config = yaml.safe_load(file_data)
def get_headers(config):
    headers = {
        "Cookie": config['cookie'],
        "User-Agent": config['user-agent']
    }
    return headers

pd.set_option('max_colwidth', 1000)
article_list = pd.read_csv('../wechat/article_list2.csv', encoding='GB18030')
headers = get_headers(config)

for index, row in article_list.iterrows():
    print("------------------------------------------------------------------------")
    print(index)
    create_time = time.strftime("%Y-%m-%d",time.localtime(row["create_time"]))
    url = row['link'].replace("\/","/")
    article = requests.get(url,headers=headers)
    if article.status_code == 200:
        html = article.text
    # for_code 
    html = html.replace("<br  />","\n")
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find(id='img-content')
    #有些文章已经被删除了，就提取不到content了
    if content:
        title = content.find(class_='rich_media_title').text.strip()
        
        # \/:*?"<>|这些字符不能作为文件名或者文件夹名
        filename = title.replace('\/','-').replace('\\','-').replace('\"','-').replace('|','-').replace(":","_").replace("?","_").replace("*","_").replace("<","_").replace(">","_") + '.md'
        try:
            copyright = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta icon_appmsg_tag appmsg_title_tag weui-wa-hotarea').text
        except:
            copyright = None
        try:
            author = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta_link weui-wa-hotarea').text
        except:
            author = content.find(class_ = "rich_media_meta rich_media_meta_text").text.split("\n")[-2].split(" ")[-1]
        blog_name = content.find(class_='profile_nickname').text
        blog_sign = content.div.div.div.find_all('p')[1].span.text
        for i in soup.html.body.find_all('script'):
            if 'publish_time' in str(i):
                publish_time = int(i.text.split("document.getElementById")[0].split("{e(")[-1].split(",\"")[-1].split("\",")[0])
        with open("../wechat2/"+filename, 'r', encoding='utf8') as file:
            mdText = ''
            mdText = mdText.join(file.readlines())
        
        mdText = mdText.replace("assets/images/", "../assets/images/")
        head = "---\nlayout: post\ntitle: \""
        head = head + mdText.split("\n")[1][3:]+"\"\nauthor: Luna"
        head = head + "\ncategories: []\ntags: []\nimage: "
        
        
        if len(mdText.split("assets/images/"))>1:
            image = mdText.split("assets/images/")[1]
            if "style" in image:
                image_link = "assets/images/" + image.split("\" style=\"")[0]
            else:
                image_link = "assets/images/" + image.split(")\n")[0]
        else:
            image_link = "assets/images/" + filename + "/0.png"
        
        head = head + image_link+"\n---\n"
        mdText = mdText.replace("\n"+mdText.split("\n")[1]+"\n", head)

        with open("../wechat_to_github/_posts/"+filename, 'w', encoding='utf8') as file:
            file.write(mdText)

In [ ]:
#给文件名加上时间
import pandas as pd
from newspaper import Article
import html2text as ht
import yaml
import time
import requests
from bs4 import BeautifulSoup
import os
import tomd
import re
from datetime import datetime

with open("../wechat/chaos-gravity.yaml", "r") as file:
    file_data = file.read()
config = yaml.safe_load(file_data)
def get_headers(config):
    headers = {
        "Cookie": config['cookie'],
        "User-Agent": config['user-agent']
    }
    return headers

pd.set_option('max_colwidth', 1000)
article_list = pd.read_csv('../wechat/article_list2.csv', encoding='GB18030')
headers = get_headers(config)

for index, row in article_list.iterrows():
    print("------------------------------------------------------------------------")
    print(index)
    create_time = time.strftime("%Y-%m-%d",time.localtime(row["create_time"]))
    url = row['link'].replace("\/","/")
    article = requests.get(url,headers=headers)
    if article.status_code == 200:
        html = article.text
    # for_code 
    html = html.replace("<br  />","\n")
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find(id='img-content')
    #有些文章已经被删除了，就提取不到content了
    if content:
        title = content.find(class_='rich_media_title').text.strip()
        print(title)
        # \/:*?"<>|这些字符不能作为文件名或者文件夹名
        filename = title.replace('\/','-').replace('\\','-').replace('\"','-').replace('|','-').replace(":","_").replace("?","_").replace("*","_").replace("<","_").replace(">","_") + '.md'
        try:
            copyright = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta icon_appmsg_tag appmsg_title_tag weui-wa-hotarea').text
        except:
            copyright = None
        try:
            author = content.find(class_='wx_tap_link js_wx_tap_highlight rich_media_meta_link weui-wa-hotarea').text
        except:
            author = content.find(class_ = "rich_media_meta rich_media_meta_text").text.split("\n")[-2].split(" ")[-1]
        blog_name = content.find(class_='profile_nickname').text
        blog_sign = content.div.div.div.find_all('p')[1].span.text
        for i in soup.html.body.find_all('script'):
            if 'publish_time' in str(i):
                publish_time = int(i.text.split("document.getElementById")[0].split("{e(")[-1].split(",\"")[-1].split("\",")[0])

        newfilename = str(datetime.fromtimestamp(publish_time)).split(" ")[0]+"-"+filename
        try:
            os.rename("_posts/"+filename, "_posts/"+newfilename)
        except:
            print(newfilename)

In [29]:
#改成Jekyll格式
import os 
import re
for item in os.listdir("../_posts/"):
    if item.split(".")[-1] == "md":
        print(item)
        with open("../_posts/"+ item, 'r', encoding='utf8') as file:
            print("-------------------------------------------------")
            mdText = ''
            mdText = mdText.join(file.readlines())
            pattern = re.compile(r'..\|..')
            pattern2= re.compile(r'\*\*.\*\*')
            a = re.findall(pattern, mdText)
            b = re.findall(pattern2, mdText)
            print(a)
            #print(b)
            #mdText.replace("|","\|")
            #with open("../_posts/"+ item, 'w', encoding='utf8') as file:
            #    file.write(mdText)

2021-10-18-Facebook自监督学习Visual Transformers(ViT)的训练经验(Moco v3) -- 训练代码解析.md
-------------------------------------------------
[]
2020-06-28-换个思路实现人工智能_ 在视觉环境中用因果归纳完成目标导向的任务——生成数据.md
-------------------------------------------------
[]
2020-03-29-换个思路实现人工智能_ 在视觉环境中用因果归纳完成目标导向的任务(上).md
-------------------------------------------------
[]
2021-09-06-Stanford CS224N笔记：词向量和词义（Word Vectors and Word Senses）.md
-------------------------------------------------
['o\\|c)', 'c\\|o)', 'o\\|c)', 'c\\|o)', 'o\\|c)', 'c\\|o)']
2020-11-23-关系归纳偏置，深度学习，和图网络.md
-------------------------------------------------
[]
2021-11-23-Facebook DeiT(Data-efficient Image Transformers) 解析.md
-------------------------------------------------
[]
2022-05-06-UC Berkeley非监督学习--Latent Variable Models -- VAE（潜变量模型--VAE）.md
-------------------------------------------------
['x\\|z)', 'x\\|z)', '2\\|x1', 'x\\|z)', 'x\\|z)', 'z\\|x)', 'x\\|z)', '(z|x)', '(z|x)']
2021-03-17-MIT因果迷你课笔记 — 因果归纳和机器学习之半监督学习.md
-----------------------